In [11]:
import spotipy.util as util
import requests

In [5]:
with open('settings/webbify_credentials.txt', 'r') as tfile:
    fileContents = tfile.read().split('\n')
    CLIENT_ID = fileContents[0].split(':')[1][1:]
    CLIENT_SECRET = fileContents[1].split(':')[1][1:]
    
# Set environment variables
AUTH_ARGS = {
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'redirect_uri': 'https://localhost:8888/'
}

In [6]:
USERNAME = 'nghokui@gmail.com'
token = util.prompt_for_user_token(USERNAME, [], **AUTH_ARGS)

token



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=d5eba668c908462aa3183670fe307302&response_type=code&redirect_uri=https%3A%2F%2Flocalhost%3A8888%2F in your browser


Enter the URL you were redirected to: https://localhost:8888/?code=AQCDp6CAV85RFbID9b72TUkOcfgU3QXuaNZMfiyGaYUuKtHIqWpUH23aeJoo3uqUkpQuJEPdZKqf823cdUOYr813DBSakeMG47KPrpcs7yVtEzCpL32DPsVhEjXVGE_PFYZ2KG9l95H3ibDidOyyrHwQH2KwLYVZYWpsX0vh8lU9nnue2lP6jch0syq6




'BQD8aMnGSabokpwro9lSzFEMtL0aJzjwmuK4y4Qh1T3oguZXhh31min7Z5d6zAHp4v2pyaE0b-dG59vFeYYUrUltbCF-GFrgDoKGcFyhvJ37BJvVaSAvez6KQ_hMs2qKHOZK_zzwx54MxA'

In [10]:
sp = spotipy.Spotify(auth=token)
playlists = sp.user_playlists('nghokui')
for playlist in playlists['items']:
    if playlist['owner']['id'] == 'nghokui':
        print(playlist['name'])

Piano & Soundtracks
Piano
PCPv
pop punk rock
things
SWAMP ASS.
BLACK PANTHER
SYBIL YEAR 1 THROWBACK
makin' me dance
country
drive
OST Selects
jazz selects
GOTG Vol. 2
RUN
other things
moombahton & co
POP PUNK ROCK
Top Songs 2016
rap/hip-hop
CALIFORNIA
JAKUBI
ANDERSON .PAAK
MARGARET GLASPY
ELÈNNE
JOEY BADA$$
MISTERWIVES
ANA TIJOUX
FILOUS
THE INTERRUPTERS
DIPLO COLLABS
CATFISH AND THE BOTTLEMEN
CARDIKNOX
KYGO
ST. LUCIA
CHAOS CHAOS
MACHINEHEART
KANYE WEST
CHANCE THE RAPPER
RIA MAE
YUNA
MAJOR LAZER
CHEAT CODES/CHAINSMOKERS/MATOMA
SMALLPOOLS
PVRIS


In [14]:
URL_BASE = 'https://api.spotify.com'
HEADERS = {'Authorization': 'Bearer {}'.format(token)}

data = requests.get('{}/v1/me'.format(URL_BASE), headers=HEADERS)
data

<Response [200]>

In [17]:
data.json()

{'display_name': 'Brian Ng',
 'external_urls': {'spotify': 'https://open.spotify.com/user/nghokui'},
 'followers': {'href': None, 'total': 20},
 'href': 'https://api.spotify.com/v1/users/nghokui',
 'id': 'nghokui',
 'images': [{'height': None,
   'url': 'https://scontent.xx.fbcdn.net/v/t1.0-1/c53.0.320.320a/p320x320/19756776_3497275234192_4181604532212724762_n.jpg?_nc_cat=107&_nc_ohc=g_rgJqe7764AQn_kHJf_uV5uvaa2THEmQ-_4hQmkjgXpgj1db7BlfPHpg&_nc_ht=scontent.xx&oh=9e3029f81d892164f886b92ab9e22660&oe=5E7F1464',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:nghokui'}